<a href="https://colab.research.google.com/github/kittimaxz/Study/blob/main/Test_CronJob.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install google-analytics-data
!pip install mysql-connector-python

In [2]:
from google.analytics.data_v1beta import BetaAnalyticsDataClient
from google.oauth2 import service_account
from google.analytics.data_v1beta.types import DateRange, Metric, Dimension, RunReportRequest
import mysql.connector
import os

api-ga4@test-api-ga4-435907.iam.gserviceaccount.com

In [3]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = r'C:\Users\VCB\Downloads\test-api-ga4-435907-52b9f02d6a3f.json'

In [4]:
# สร้าง Client เพื่อเชื่อมต่อกับ GA4 API
client = BetaAnalyticsDataClient()
client

DefaultCredentialsError: File test-api-ga4-435907-52b9f02d6a3f.json was not found.

In [ ]:
# กำหนดการร้องขอข้อมูล
request = RunReportRequest(
    property="properties/307754729",  # ใส่ Property ID ของ GA4
    dimensions=[Dimension(name="country")],
    metrics=[Metric(name="activeUsers")],
    date_ranges=[DateRange(start_date="2023-01-01", end_date="today")]
)
request

In [ ]:
# รันการดึงข้อมูล
response = client.run_report(request)

In [ ]:
# พิมพ์ข้อมูลออกมา
for row in response.rows:
    print(row)

In [ ]:
# เชื่อมต่อกับ MySQL
connection = mysql.connector.connect(
    host="159.138.255.180",
    user="root",
    password="Vcb168899@",
    database="vcb_ba2"
)
connection

In [ ]:
cursor = connection.cursor()
cursor

In [ ]:
sql = "INSERT INTO test_backup_ga4 (country, active_users) VALUES (%s, %s)"
sql

In [ ]:
for row in response.rows:
    data = (row.dimension_values[0].value, row.metric_values[0].value)
    cursor.execute(sql, data)

In [ ]:
connection.commit() # save การเปลี่ยนแปลง
connection.close() # ปิดการเชื่อมต่อ